<a href="https://colab.research.google.com/github/AAhmadS/NLP_HW1/blob/main/NLP_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Natural Language Processing-Homework1**

####**Student name** : Amirahmad Shafiee<br/>**Student number** : 99104027<br/>**Chosen task** : classification
In this notebook we aim to do common but rather essential preprocess actions on some text chosen from the persian literature world. After doing so, an NLP task would be operated on the processed text.

In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/NLP

/content/gdrive/.shortcut-targets-by-id/1Ikn75GANDVFyb0klJe4-EBQJikt4_Ob_/NLP


##Essential packages and data:

The data we tend to use in this notebook will be the texts extracted from philosophers books on ethics, like : 
>**Immanuel Kant's** "Critique of Pure Reason"<br/>
>**Friedrich Neitzsche's**"Human all too human"<br/>
>**Georg Wilhelm Friedrich Hegel's**"Phenomenology of spirit"<br/>

In [ ]:
!gdown https://rauterberg.employee.id.tue.nl/lecturenotes/DDM110%20CAS/Kant-1781%20Critique%20of%20Pure%20Reason.pdf
!gdown http://www.faculty.umb.edu/gary_zabel/Courses/Marxist_Philosophy/Hegel_and_Feuerbach_files/Hegel-Phenomenology-of-Spirit.pdf
!gdown https://drive.google.com/file/d/1x4cM62wac-nEU8X_rEyYLLsWOX67Kqmr/view?usp=share_link

Our data comes in the pdf form. So we need the primary text extractors compatible with pdf format.

In [3]:
!pip install -q openpyxl==3.0.10
!pip install -q PyPDF2==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 12.9 MB/s eta 0:00:00


In [4]:
import random 
import tqdm
import nltk, re
from nltk import word_tokenize, sent_tokenize, WhitespaceTokenizer, FreqDist, pos_tag
from nltk.stem import WordNetLemmatizer, PorterStemmer

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist

import itertools
import string
import numpy as np
import pandas as pd

from tqdm import tqdm
import pickle

import plotly.express as px
import plotly.graph_objects as go

from PyPDF2 import PdfReader

In [5]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

##Section A: Extracting data



In [ ]:
mask_list1 = np.concatenate((np.arange(36),np.array([529]),np.arange(530,631)))
with open('pdf_mask_list_heg.npy','wb') as f:
  np.save(f,mask_list1)

mask_list2 = np.concatenate((np.arange(103),np.array([129,156,625]),np.arange(705,785)))
with open('pdf_mask_list_kan.npy','wb') as f:
  np.save(f,mask_list2)

mask_list3 = np.concatenate((np.arange(30),np.array([230,231,232,233,325]),np.arange(421,425)))
with open('pdf_mask_list_nei.npy','wb') as f:
  np.save(f,mask_list3)

In [6]:
name_list=["heg","kan","nei"]
address_list=["Hegel-Phenomenology-of-Spirit.pdf","Kant-1781%20Critique%20of%20Pure%20Reason.pdf","Human, All Too Human_ A Book for Free Spirits - PART I + PART II - Friedrich Nietzsche - PDF.pdf"]

In [ ]:
for i in range(3):
  reader = PdfReader(address_list[i])
  number_of_pages = len(reader.pages)

  with open(f'pdf_mask_list_{name_list[i]}.npy','rb') as f:
    mask_list = np.load(f)
    text_pages_index = np.delete(np.arange(number_of_pages),mask_list)

  text = ""
  for ind in text_pages_index:
    text+=reader.pages[int(ind)].extract_text()

  with open(f'final_text_{name_list[i]}.txt','wb') as f:
    pickle.dump(text,f)  

In [7]:
texts = []
for x in name_list:
  with open(f'final_text_{x}.txt','rb') as f:
    text = pickle.load(f)
    texts.append(text)

In [8]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

sentences = [sent_detector.tokenize(text) for text in texts]

In [9]:
data = pd.DataFrame()
labels=np.array([])
labels_auth = []
for i in range(3):
  data = data.append(sentences[i],ignore_index = True)
  labels = np.concatenate((labels, (i*np.ones(len(sentences[i])))))

labels = labels.astype(np.int)
labels_auth = [name_list[i] for i in labels]
data.rename(columns = {0 : "sentences"}, inplace=True)
data["labels"] = labels
data["author"] = labels_auth

<ipython-input-9-605b0474b0a1>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(sentences[i],ignore_index = True)
<ipython-input-9-605b0474b0a1>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  labels = labels.astype(np.int)


In [11]:
data.sample(5)

,sentences,labels,author
19435,-A good educator knows cases in which he is pr...,2,nei
6698,"In the mystery Df \nbread and wine, it appropr...",0,heg
13328,A55I/B 579 Suppose now that one could say reas...,1,kan
6151,"But, in this picture-thinking, reaHtydoes \nno...",0,heg
20156,"Further stages of mor­\nality, and thus of mea...",2,nei


In [12]:
for txt,x,sen in zip(texts,name_list,sentences):
  print(f"{x} text consists of {len(txt)} characters, in other words the volume of the text is about {len(txt)/1e6:.2f} megs\nalso {len(sen)} sentences have been spotted")

heg text consists of 1251449 characters, in other words the volume of the text is about 1.25 megs
also 7317 sentences have been spotted
kan text consists of 1642328 characters, in other words the volume of the text is about 1.64 megs
also 8148 sentences have been spotted
nei text consists of 1102859 characters, in other words the volume of the text is about 1.10 megs
also 5997 sentences have been spotted


In [41]:
class distributions:

  def minmax(data, col_end = 'sentence_len_by_words', col_beg = 'sentences'):
    data[col_end] = data[col_beg].apply(lambda t: len(word_tokenize(t)))

    min_max_len = data[col_end].min(), data[col_end].max()
    print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

    return

  def word_freq(data, col_end = "sentence_len_by_words"):

      fig = go.Figure()

      fig.add_trace(go.Histogram(
          x=data[col_end]
      ))

      fig.update_layout(
          title_text='Distribution of word counts within data',
          xaxis_title_text='Word Count',
          yaxis_title_text='Frequency',
          bargap=0.2,
          bargroupgap=0.2)

      fig.show()

      return

  def data_gl_than(data, less_than=25.0, greater_than=3.0, col='sentence_len_by_words'):
      data_length = data[col].values

      data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

      data_glt_rate = (data_glt / len(data_length)) * 100

      print(f'sentences with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

      return

  def full_df_run(data, col_begin = 'sentences'):
    distributions.minmax(data)
    distributions.word_freq(data)
    distributions.data_gl_than(data)


  def word_and_freq(data, col = "tokenized_sents"):
    tokenized_sent = data["tokenized_sents"]
    mp_freqdist = FreqDist(itertools.chain(*tokenized_sent))
    top20words=mp_freqdist.most_common(20)
    print ('%-16s' % 'word', '%-16s' % 'Frequency','%-16s' %  '% of the total')
    for topword in top20words:
        percent=(topword[1]/len(tokenized_sent))*100
        print ('%-16s' % topword[0], '%-16s' % topword[1],'%-16s' %  percent)
    
    return

In [14]:
distributions.full_df_run(data)

Min: 1 	Max: 340


sentences with word length of greater than 3.0 and less than 25.0 includes 31.83% of the whole!


which clearly is not a result we were looking forward to have, next we will clean our text, numbers and oddly nonsentences considered as one, will be eliminated.

##Section B: Text preprocessing

###Subsection 1: tokenizing

In [15]:
tokenized_sents = [word_tokenize(sent) for sent in data["sentences"]]
data["tokenized_sents"] = tokenized_sents
data.sample(5)

,sentences,labels,author,sentence_len_by_words,tokenized_sents
11781,"According ly, \na Object \nb Verba/tnisses \n4...",1,kan,12,"[According, ly, ,, a, Object, b, Verba/tnisses..."
18293,-And what is it destines us for this almost \n...,2,nei,12,"[-And, what, is, it, destines, us, for, this, ..."
11702,Bk.,1,kan,2,"[Bk, .]"
16576,156 \nInspiration again.,2,nei,4,"[156, Inspiration, again, .]"
16770,"These, moreover, desire that their requirement...",2,nei,27,"[These, ,, moreover, ,, desire, that, their, r..."


In [42]:
distributions.word_and_freq(data,"tokenized_sents")

TypeError: ignored

###Subsection 2: normalising

In [70]:
stop_words = stopwords.words('english')
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [ ]:
def normalize_sentence(tokenized_sents, minimum_length=2, stopword_removal=True, lower_case=False, punctuation_removal=True):
  
    normalized_sents = tokenized_sents
    
    if stopword_removal:
        # Remove stopwords in English and also the given domain stopwords
        normalized_sents=[[word for word in sentence if (word.lower() not in stopwords)] for sentence in tokenized_sents ]

    if punctuation_removal:
        # Remove punctuations
        normalized_sents=[[word for word in sentence if word not in string.punctuation] for sentence in normalized_sents ]

    if lower_case:
        # Convert everything to lowercase and filter based on a min length
        normalized_sents=[[word.lower() for word in sentence if len(word)>minimum_length] for sentence in normalized_sents ]

    elif minimum_length>1:
        normalized_sents= [[word for word in sentence if len(word)>minimum_length] for sentence in normalized_sents ]        
        
    return normalized_sents

###Subsection 3: Frequency analysis

###Subsection 4: stop-word elimination

###Subsection 5: lemmatization and stemming

##Section C: POS-tagging

##Section D: author classification

In [ ]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

### Configuration

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [ ]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-taaghceh/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [ ]:
# create a key finder based on label 2 id and id to label

label2id = {label: i for i, label in enumerate(poets)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'khajoo': 0, 'asad': 1, 'bahar': 2, 'feyz': 3, 'asadi': 4, 'hafez': 5, 'jami': 6, 'kamal': 7, 'moulavi': 8, 'parvin': 9}
id2label: {0: 'khajoo', 1: 'asad', 2: 'bahar', 3: 'feyz', 4: 'asadi', 5: 'hafez', 6: 'jami', 7: 'kamal', 8: 'moulavi', 9: 'parvin'}


In [ ]:
# setup the tokenizer and configuration

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

### Input Embeddings

In [ ]:
idx = np.random.randint(0, len(train))
sample_poem = train.iloc[idx]['poem']
sample_poet = train.iloc[idx]['poet']

print(f'Sample: \n{sample_poem}\n{sample_poet}')

Sample: 
دیده مرغ صراحی بقدح باز کنیم
0


In [ ]:
tokens = tokenizer.tokenize(sample_poem)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  poem: {sample_poem}')
print(f'  Coded_tokens: {tokens}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

  poem: دیده مرغ صراحی بقدح باز کنیم
  Coded_tokens: ['دیده', 'مرغ', 'صراحی', 'بق', '##د', '##ح', 'باز', 'کنیم']
   Tokens: دیده مرغ صراحی بقدح باز کنیم
Token IDs: [4275, 6004, 96985, 8997, 2013, 2051, 3012, 3592]


In [ ]:
encoding = tokenizer.encode_plus(
    sample_poem,
    max_length=32,
    truncation=True,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt',  # Return PyTorch tensors
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[    2,  4275,  6004, 96985,  8997,  2013,  2051,  3012,  3592,     4,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])


### Dataset

In [ ]:
class PoemsDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, poems, targets=None, label_list=None, max_len=128):
        self.poems = poems
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len

        
        self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
    
    def __len__(self):
        return len(self.poems)

    def __getitem__(self, item):
        poem = str(self.poems[item])

        if self.has_target:
            target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
            poem,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
        
        inputs = {
            'poem': poem,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)
        
        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = PoemsDataset(
        poems=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len, 
        label_list=label_list)
    
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
label_list = poets
train_data_loader = create_data_loader(train['poem'].to_numpy(), train['poet'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(valid['poem'].to_numpy(), valid['poet'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test['poem'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

In [ ]:
sample_data = next(iter(train_data_loader))

print(sample_data.keys())

print(sample_data['poem'])
print(sample_data['input_ids'].shape)
print(sample_data['input_ids'][0, :])
print(sample_data['attention_mask'].shape)
print(sample_data['attention_mask'][0, :])
print(sample_data['token_type_ids'].shape)
print(sample_data['token_type_ids'][0, :])
print(sample_data['targets'].shape)
print(sample_data['targets'])

dict_keys(['poem', 'input_ids', 'attention_mask', 'token_type_ids', 'targets'])
['نهادی دو سه پیل زی شاه پی', 'نه محض جوهر روحی که روح جوهر جانی', 'ما مردانیم شسته بر تنگ دره', 'رانی ام چونان که راند بنده ای  را پادشاهی', 'ساحران در عهد فرعون لعین', 'وزو به عالم جان تحفه گونه گون آید', 'ساقی به دست باش که غم در کمین ماست', 'بسی گوهر ز بام آویختندم', 'الا یا راهب الدیر فهل مرت بک النوق', 'دریاست دهر کشتی خویش استوار دار', 'ما یا آنیم و این دگر فرع', 'بجانب تو روان بود جانم از شوقت', 'وان جامه و رخت تو سخت نغز است', 'چنین گفت کای بخت پیشت رهی', 'چو رامین شد به بند مهر بسته', 'تا کشی گوهری از مخزن غیب']
torch.Size([16, 128])
tensor([    2, 13460,  2897,  3140, 14655,  3029,  3603,  3063,     4,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,  

In [ ]:
sample_test = next(iter(test_data_loader))
print(sample_test.keys())

dict_keys(['poem', 'input_ids', 'attention_mask', 'token_type_ids'])


In [ ]:
class ClassifierModel(nn.Module):

    def __init__(self, config):
        super(ClassifierModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, 10)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        pooled_output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids)['pooler_output']
        
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits 

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

In [ ]:
pt_model = ClassifierModel(config=config)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pt_model <class '__main__.ClassifierModel'>


In [ ]:
for name,param in pt_model.named_parameters():
  if'classifier' not in name:
    param.requires_grad = False
  else:
    print(name)

classifier.weight
classifier.bias


### Training

In [ ]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)

    return y, loss


def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):
            
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            # calculate the batch loss
            loss = loss_fn(outputs, targets)

            # accumulate all the losses
            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)
    
    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss


def train_op(model, 
             data_loader, 
             loss_fn, 
             optimizer, 
             scheduler, 
             step=0, 
             print_every_step=100, 
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None, 
             clip=0.0):
    
    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        # move tensors to GPU if CUDA is available
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        # clear the gradients of all optimized variables
        optimizer.zero_grad()

        # compute predicted outputs by passing inputs to the model
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
        # convert output probabilities to predicted class
        _, preds = torch.max(outputs, dim=1)

        # calculate the batch loss
        loss = loss_fn(outputs, targets)

        # accumulate all the losses
        losses.append(loss.item())

        # compute gradient of the loss with respect to model parameters
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        # perform optimization step
        optimizer.step()

        # perform scheduler step
        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)
        
        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))
            
            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss
        
        return eval_loss_min


    return eval_cb

In [ ]:
from torch import optim

optimizer = optim.Adam(pt_model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model, 
        data_loader=train_data_loader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader, 
        clip=CLIP)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=pt_model, 
        data_loader=valid_data_loader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/3772 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Epoch: 1/3...Step: 1000...Train Loss: 2.304766...Train Acc: 0.116...Valid Loss: 2.284853...Valid Acc: 0.136...
Validation loss decreased (inf --> 2.284853).  Saving model ...


Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

...

Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Epoch: 2/3...Step: 4000...Train Loss: 2.256819...Train Acc: 0.159...Valid Loss: 2.244747...Valid Acc: 0.181...
Validation loss decreased (2.254539 --> 2.244747).  Saving model ...
Epoch: 2/3...Step: 4000...Train Loss: 2.256819...Train Acc: 0.159...Valid Loss: 2.244747...Valid Acc: 0.181...
Validation loss decreased (2.254539 --> 2.244747).  Saving model ...


KeyboardInterrupt: ignored

KeyboardInterrupt: ignored

### Prediction

In [ ]:
def predict(model, comments, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size, None)
    
    predictions = []
    prediction_probs = []

    
    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            
            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [ ]:
test_poems = test['poem'].to_numpy()
preds, probs = predict(pt_model, test_poems, tokenizer, max_len=128)

print(preds.shape, probs.shape)

  0%|          | 0/590 [00:00<?, ?it/s]

(18860,) (18860, 10)


In [ ]:
preds

array([9, 5, 5, ..., 3, 4, 6])

In [ ]:
preds_label = [id2label[t] for t in preds]
preds_label[0:5]

['parvin', 'hafez', 'hafez', 'asad', 'asadi']

##F1 Score and report

In [ ]:
y_test, y_pred = test['poet'].values, preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print("--------------classification_report---------------")
print(classification_report(y_test, y_pred, target_names=label_list))

F1: 0.18183054145230343
--------------classification_report---------------
              precision    recall  f1-score   support

      khajoo       0.19      0.06      0.09      1886
        asad       0.17      0.23      0.19      1886
       bahar       0.18      0.21      0.19      1886
        feyz       0.20      0.20      0.20      1886
       asadi       0.28      0.41      0.33      1886
       hafez       0.18      0.26      0.21      1886
        jami       0.15      0.17      0.16      1886
       kamal       0.16      0.12      0.14      1886
     moulavi       0.22      0.09      0.13      1886
      parvin       0.18      0.17      0.17      1886

    accuracy                           0.19     18860
   macro avg       0.19      0.19      0.18     18860
weighted avg       0.19      0.19      0.18     18860



In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[108 230 135 240 211 368 175 178 102 139]
 [ 42 427 178 172 363 210 164 122  41 167]
 [ 28 186 396 131 281 199 247 122  90 206]
 [ 61 231 157 370  91 361 230 155  83 147]
 [ 23 258 233  63 769  90 154 103  29 164]
 [ 79 231 181 217 104 495 202 149  80 148]
 [ 52 200 255 146 294 210 317 132  70 210]
 [ 74 261 182 222 209 282 201 229  86 140]
 [ 34 281 270 161 189 241 237 149 178 146]
 [ 53 242 263 138 199 292 202 131  48 318]]
